In [9]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/BN4101 Final Product/BN4101-Final-Product/

!pip install mediapipe

#------------------------SQL-----------------------------------------
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/

%sql CREATE DATABASE predictionbase
%sql USE predictionbase

## creating the 'all_predictions' table again with the 'PRIMARY KEY'
## prediction level from 0 to 3
%sql CREATE TABLE all_predictions (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, staff_id VARCHAR(255), \
student_id VARCHAR(255), module_code VARCHAR(255), class_id VARCHAR(255), frame_num INT(11), pred_lvl INT(11))

#-------------------------------------------------------------------------


import os
from os import listdir
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from google.colab import files

from IPython.display import Image
import pandas as pd
import numpy as np
import cv2
import time

import mediapipe as mp

import NusFypFeatureEngineering as nffe

from keras.models import model_from_json


def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename


mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

def imageProcess(image, subject_name, sequence, label, keypoints):

    # For static images:
    face_mesh = mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        min_detection_confidence=0.5)
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

    # Convert the BGR image to RGB before processing.
    try:
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if keypoints == {}:

            keypoints["Name"] = [subject_name]
            keypoints["Sequence"] = [sequence]
            keypoints["Target"] = [label]

            for i, data_point in enumerate(results.multi_face_landmarks[0].landmark):
                keypoints['X'+str(i)] = [data_point.x]
                keypoints['Y'+str(i)] = [data_point.y]

        else:
            for i, data_point in enumerate(results.multi_face_landmarks[0].landmark):
                keypoints['X'+str(i)] += [data_point.x]
                keypoints['Y'+str(i)] += [data_point.y]

            keypoints["Name"] += [subject_name]
            keypoints["Sequence"] += [sequence]
            keypoints["Target"] += [label]

    except:
        print("No face detected.")

        for i in range(468):
            keypoints['X'+str(i)] += ["No Face"]
            keypoints['Y'+str(i)] += ["No Face"]

        keypoints["Name"] += [subject_name]
        keypoints["Sequence"] += [sequence]
        keypoints["Target"] += [label]

    face_mesh.close()

    return keypoints

def laggingDf(df, maxLag):
    tempDf = df
    for lag in range(1, maxLag+1):
        # Lagging the dataframe and subtract from previous row
        laggedDiffDf = pd.DataFrame(tempDf["Name"]).join(tempDf.groupby(['Name']).shift(
            0).add_prefix('lag'+str(lag)+"_")).groupby(["Name"]).diff(periods=lag)
        # Adding the wrangled rows back into original dataframe
        df = df.join(laggedDiffDf[laggedDiffDf.columns[2:]])
    return df.dropna()

def mergeDf(df1, df2):
    a = np.intersect1d(fe_ele_train_dataset.columns, dfCleaned.columns)

    df1 = df1.drop(a, axis=1)
    df3 = df1.join(df2)

    return df3.dropna()

i = 0

maxLag = 1   #Looking into how many seconds we want to look backwards

#Feature Engineering preparation
x_fea,y_fea = nffe.sepXY([],[])

x_fea_eng,x_fea_alone = nffe.sepEng(x_fea,[],[])
y_fea_eng,y_fea_alone = nffe.sepEng(y_fea,[],[]) 


#Loading deep learning model
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

#Loading final filters
finalFil = pd.read_csv("final_features.csv")

#Class information
StudentID = "A0168781N"
StaffID = "00001"
ModuleID = "BN4301"
ClassID = "S1"
frame = 0

while True:
  try:
    keypoints = {}
    #Take the photo the same number of time we want to look backwards
    for i in range(maxLag+1):
      start_time = time.time()

      filename = take_photo()
      
      i += 1

      keypoints = imageProcess(cv2.imread("photo.jpg"),"Yong Jun",i,0,keypoints)

      #Wait for one second
      time.sleep(1)

    fe_ele_train_dataset = nffe.addNewFeature(pd.DataFrame(keypoints),x_fea_eng)
    fe_ele_train_dataset = nffe.addNewFeature(fe_ele_train_dataset,y_fea_eng)

    fe_ele_train_dataset = fe_ele_train_dataset[["Target"]+nffe.fea80()]

    dfCleaned = laggingDf(pd.DataFrame(keypoints),maxLag)

    talliedDf = mergeDf(fe_ele_train_dataset,dfCleaned)

    preds = loaded_model.predict(talliedDf[finalFil["0"]],batch_size = 128)

    #Insert into SQL
    values = (StaffID,StudentID,ModuleID,ClassID,frame,np.argmax(preds))

    %sql INSERT INTO all_predictions (staff_id, student_id, module_code, class_id, frame_num, pred_lvl) VALUES :values

    print("Engagement Level is",np.argmax(preds))

    frame += 1

    print((time.time()-start_time))
  
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

    print((time.time()-start_time))


 * mysql+pymysql://root:***@/
Engagement Level is 2
2.9145941734313965


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 * mysql+pymysql://root:***@/
Engagement Level is 2
2.8849692344665527


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored

In [10]:
%sql SELECT * FROM all_predictions

 * mysql+pymysql://root:***@/


,id,staff_id,student_id,module_code,class_id,frame_num,pred_lvl
0,1,00001,A0168781N,BN4301,S1,0,2
1,2,00001,A0168781N,BN4301,S1,1,2
2,3,00001,A0168781N,BN4301,S1,2,2
3,4,00001,A0168781N,BN4301,S1,3,2
4,5,00001,A0168781N,BN4301,S1,4,2


In [ ]:
%sql CREATE DATABASE predictionbase
%sql USE predictionbase

## creating the 'user_login' table again with the 'PRIMARY KEY'
%sql CREATE TABLE user_login (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, user_name VARCHAR(255), \
password VARCHAR(255), staff_id VARCHAR(255), staff_name VARCHAR(255))

## creating the 'student_db' table again with the 'PRIMARY KEY'
%sql CREATE TABLE student_db (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, student_id VARCHAR(255), \
student_name VARCHAR(255), module_code VARCHAR(255), class_id VARCHAR(255))

## creating the 'module_db' table again with the 'PRIMARY KEY'
%sql CREATE TABLE module_db (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, staff_id VARCHAR(255), \
module_code VARCHAR(255), class_id VARCHAR(255))

## creating the 'all_predictions' table again with the 'PRIMARY KEY'
## prediction level from 1 to 4
%sql CREATE TABLE all_predictions (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, staff_id VARCHAR(255), \
student_id VARCHAR(255), module_code VARCHAR(255), class_id VARCHAR(255), frame_num INT(11), pred_lvl INT(11))

 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/


""


In [ ]:
import pymysql.cursors

values = [
    ("A0172303U", "A1234567B", "BN1102", "S1", 1, 4),
    ("A0172303U", "A1234567B", "BN1102", "S1", 2, 4),
    ("A0172303U", "A1234567B", "BN1102", "S1", 3, 3),
    ("A0172303U", "A1234567B", "BN1102", "S1", 4, 1),
    ("A0172303U", "A234567N", "BN1102", "S1", 1, 4),
    ("A0172303U", "A234567N", "BN1102", "S1", 2, 1),
    ("A0172303U", "A234567N", "BN1102", "S1", 3, 2),
    ("A0172303U", "A234567N", "BN1102", "S1", 4, 3),
    ("A0172303U", "A500000A", "BN1102", "S1", 1, 4),
    ("A0172303U", "A500000A", "BN1102", "S1", 2, 1),
    ("A0172303U", "A500000A", "BN1102", "S1", 3, 2),
    ("A0172303U", "A500000A", "BN1102", "S1", 4, 3)
   
]

for value in values:
  %sql INSERT INTO all_predictions (staff_id, student_id, module_code, class_id, frame_num, pred_lvl) VALUES :value


 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/


In [ ]:
%sql SELECT * FROM all_predictions

 * mysql+pymysql://root:***@/


,id,staff_id,student_id,module_code,class_id,frame_num,pred_lvl
0,1,A0172303U,A1234567B,BN1102,S1,1,4
1,2,A0172303U,A1234567B,BN1102,S1,2,4
2,3,A0172303U,A1234567B,BN1102,S1,3,3
3,4,A0172303U,A1234567B,BN1102,S1,4,1
4,5,A0172303U,A234567N,BN1102,S1,1,4
5,6,A0172303U,A234567N,BN1102,S1,2,1
6,7,A0172303U,A234567N,BN1102,S1,3,2
7,8,A0172303U,A234567N,BN1102,S1,4,3
8,9,A0172303U,A500000A,BN1102,S1,1,4
9,10,A0172303U,A500000A,BN1102,S1,2,1
